Import all required packages.
---

Note: The ase.py file of the tblite module is not up to date yet in the official version of the package. Until this is the case, please change it as given in this folder if you want to use xTB.

In [44]:
import os
import ase.io
from tblite.ase import TBLite
from adaptive_sampling.interface import AseMD
from adaptive_sampling.exploration_tools import Hyperreactor
from adaptive_sampling.units import kB_in_atomic
from adaptive_sampling.exploration_tools.utils import spheric_fib_init

Set up simulation parameters.
---------

In [ ]:
jobname        = "test"

# MD Parameters
mute             = 1                      # controls the amount of MD ouput in the notebook/terminal
out_freq         = 5                      # output every 50 steps
seed             = 42                     # seed for random initialization of momenta

totalt           = 5e3                    # production length [fs]
dt               = 0.5e0                  # time step [fs]
nsteps           = int(totalt/dt) + 1     # total number of MD production steps (add 1 to account for step 0)
heat_steps       = 100                    # total number of MD heating steps

langevin_thermos = True                   # Langevin thermostat
init_temp        = 1.0                    # initial temperature
target_temp      = 298.15                 # target temperature
friction_const   = 7.0e-3                 # friction constant [1/fs]

In [ ]:
# Molecule
HCN_mol = ase.io.read("data/hcn_opt.xyz")

# for spheric_fib_init you have to initialize each molecule type as ASE Atoms like above for HCN and then provide them in a list
# the second list contains the number of each type of molecule
# the third element is the radius between the subshells, usually 3 Angstroms is a good choice
atoms, radius    = spheric_fib_init([HCN_mol], [50], 3)                                      # Fibonacci initalization
mult             = 1                                                                         # multiplicity
charge           = 0                                                                         # charge
atoms.calc       = TBLite(method="GFN2-xTB", verbosity=0, charge=charge, multiplicity=mult)  # add calculator

In [ ]:
# Spherical Constraint

k_conf           = 1.0                           # force constant for spherical confinement [kcal/(mol*A^2)
r_max            = 15.0                          # maximal radius for spherical confinement [A]
r_min            = 7.5                          # minimal radius for spherical confinement [A]
period           = 500                          # period for the smooth-step spherical confinement [fs]

# Hyperdynamics Constraint
init_steps      = 100                            # initalization steps for hyperdynamics -> the potential is not biased here
equil_steps     = 1000                           # the equilibration steps include the init_steps and the potential is biased while still updating the potential distributions
aMD_parameter   = 10.0*kB_in_atomic*target_temp  # alpha/sigma0 depending on the chosen HRD (10*k_B*T is usually a good choice for sigma0) [E_h]

the_md = AseMD(                                  # initialization of the MD object
        atoms=atoms,
        dt=dt,
        thermostat=True,
        target_temp=target_temp,
        friction=friction_const,
        seed=seed,
        mute=mute,
    )

the_bias = Hyperreactor(                        # initialization of the bias object
        md=the_md,
        mode="GaHRD_lower",
        amd_parameter=aMD_parameter,
        init_steps=init_steps,
        equil_steps=equil_steps,
        equil_temp=target_temp,
        r_max=r_max,
        r_min=r_min,
        k_conf=k_conf,
        t_total=period,
    )

In [ ]:
print("========================================================================================")
print("                             XTB and MD Object Initialized                            \n")

Start heating.
--------------

In [ ]:
the_md.calc_init(init_temp=init_temp)      # initial step for heating
# first heat
the_md.heat(nsteps=heat_steps,             # heating with momenta rescaling
        start_temp=init_temp,
        target_temp=target_temp,
        stages=10,
        out_freq=out_freq,
        prefix="heat_"+jobname
    )

print("========================================================================================")
print("                                 Heating Completed                                    \n")

Check chosen parameters.
---------------------

In [ ]:
print("========================================================================================")
print("                                 Exploration Parameters                               \n")
print("r_max:\t%14.3f A"%r_max)
print("r_min:\t%14.3f A"%r_min)
print("k_conf:\t%14.3f kcal/(mol*A^2)"%k_conf)
print("period:\t%14.3f ps"%(period/1000))
print("init_steps:\t%6i"%init_steps)
print("equil_steps:\t%6i"%equil_steps)
print("sigma:\t%14.6f E_h"%aMD_parameter)

Run exploration MD.
------------------

In [ ]:
print("========================================================================================")
print("                                 Exploration started                                  \n")

the_md.set_explorationpots(the_bias)                                          # adding the bias after the heating
the_md.calc_init(init_temp=target_temp, time_reset=True)                            # initial step of the production
the_md.run(nsteps=nsteps, out_freq=out_freq, prefix="production_"+jobname)    # MD loop

In [ ]:
print ("MD Done")